In [ ]:
import os

from datetime import datetime
import matplotlib.pyplot as plt

import pandas as pd
from IPython.display import display,Markdown
import matplotlib.dates as mdates

saveFig = True
nsets = {
    'Central Station Taxi Rank':[1409, 1410],  #
    'Near Central Station': [1408, 1414],  #
    'Newcastle Helix':[2604, 2606, 2603, 2602, 2605, ],  #
    'Gosforth High Street':[2753, 2752, ],  # nr gosforth high street
    'Regent Centre Interchange':[2902, 2904, 2903, ],  # regent centre metro + bus stop (3/6)
    'Symphony Court, Durham Road':[1207, 1206, 1202, ],  #
    "St George's Church, Low Fell":[1006, 1007, ], # 1002, ],  #
   # 'Westgate Rd - St James Blvd Intersection':[1702, ], # 1701 #
}



# Noise levels across Newcastle upon Tyne

The data represents daily median levels of noise, dervived from sensors deployed around Newcastle. Multiple sensors are combined when they are close.

In [ ]:

nmin = 30

colourUp = '#f64a8a'
colourDown = '#233067'
resample_folder_path = os.path.join('../cache/','noise', 'processed')

for title,nset in nsets.items():
    frames = []
    for i,sensor_id in enumerate(nset):
        sensor_name = f'PER_EMOTE_{sensor_id}'
        df = pd.read_pickle(os.path.join(resample_folder_path,
                                         'noise-data-{sensor_name}.pkl'.format(sensor_name=sensor_name)))
        if ((df['data_prop']< 0.3).value_counts()[False]/len(df.index)) >= 0.85:
            frames.append(df)
    if frames:
        fig, plots = plt.subplots(
             len(frames), 1,
             figsize = (18, 1.5 * len(frames)),
            dpi=300
        )
        # # Or equivalently,  "plt.tight_layout()"
        fig.tight_layout()
        display(Markdown('## '+title))

    if len(frames) == 1:
        plots = [plots]


    for i,df in enumerate(frames):

        df = df[df['data_prop'] > 0.3]

        df = df[df.index >= datetime(2020,1,1)]

        dfWeekdays = df[df.index.to_series().apply(
            lambda t: t.strftime('%A') not in ['Saturday', 'Sunday']
        )]
        dfWeekends = df[df.index.to_series().apply(
            lambda t: t.strftime('%A') in ['Saturday', 'Sunday']
        )]

        plots[i].bar(
            dfWeekdays.index,
            dfWeekdays['Value'],
            color=colourDown,
            label='Noise level on weekdays'
        )
        plots[i].bar(
            dfWeekends.index,
            dfWeekends['Value'],
            color=colourUp,
            label='Noise level on weekends'
        )
        if i==0:
            plots[i].legend(
                loc='upper right',
                ncol=2,
                fontsize=11,
                frameon=False,
                bbox_to_anchor=(1.0, 1.35)
            )
        first = False
        plots[i].set_ylim([55,80])
        monthyearFmt = mdates.DateFormatter('%b %Y')
        plots[i].xaxis.set_major_formatter(monthyearFmt)
        plots[i].set_xlim([datetime(2020,1,1),datetime.now()])

    plt.show()

